In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
wig_df = pd.read_csv('wig.csv')

In [3]:
wig_df['day_change'] = wig_df['Zamkniecie'] / wig_df['Zamkniecie'].shift(-1)

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1991-04-16,1000.00,1000.00,1000.00,1000.00,325.0
1,1991-04-23,967.70,967.70,967.70,967.70,5905.0
2,1991-04-30,945.60,945.60,945.60,945.60,7162.0
3,1991-05-14,939.60,939.60,939.60,939.60,18300.0
4,1991-05-21,966.10,966.10,966.10,966.10,14750.0
5,1991-05-28,956.50,956.50,956.50,956.50,31440.0
6,1991-06-04,969.40,969.40,969.40,969.40,12396.0
7,1991-06-11,960.40,960.40,960.40,960.40,26247.0
8,1991-06-18,866.80,866.80,866.80,866.80,33496.0
9,1991-06-25,850.20,850.20,850.20,850.20,25264.0
